# **Tutorial 12**


### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import io
import scipy.stats as stats
import seaborn as sns
from sklearn.linear_model import LinearRegression #for linear regression model
import random as rd #random

### Spikes Example

loading the data for the spikes example (in the tutorial folder)


In [ ]:
#upload and read from PC/Drive
# after running press 'Choose File'

uploaded = files.upload()

data = pd.read_csv(io.BytesIO(uploaded['numSpikes.csv']))

for col in data.columns: # see names of the columns in a loop
    print(col)

print(data)

In [ ]:
# plot original data, calculate mean
spikes_data = data.numSpikes
data_len = len(spikes_data)
data_mean = data.mean()

plt.hist(spikes_data, bins = 50)
plt.plot([data_mean, data_mean] , plt.gca().get_ylim(), color = 'lime', lw = 3, label = f'Mean = {np.round(data_mean[0], 2)}')
plt.xlabel('Number of spikes per second', fontsize = 14)
plt.ylabel('Number of occurances', fontsize = 14)
plt.title('Neuron Activity', fontsize = 14)
plt.legend(fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("spikes_hist.png", bbox_inches='tight')
files.download("spikes_hist.png")

In [ ]:
#figure with the distribution
plt.hist(spikes_data, bins=np.arange(13)-0.5, rwidth = 0.2, density = True, label = 'data')

x = np.arange(0, 15, 1)
p = stats.poisson.pmf(x, data_mean)
plt.plot(x, p, 'lime', linewidth=3, label = 'Poisson Distribution Fit')
title = "Poisson ( lambda = %.2f )" % (data_mean)
plt.title(title)
plt.xlabel('Number of spikes per second', fontsize = 14)
plt.ylabel('Density', fontsize = 14)
plt.legend(loc = 'upper right', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("spikes_fitpois.png", bbox_inches='tight')
files.download("spikes_fitpois.png")

#### Parametric Bootstrap

In [ ]:
# Logical Distribution

num_boot = 2000 #number of bootstrap repititions
boot_data = np.random.poisson(data_mean, size = (data_len, num_boot) ) #select the same sample size as the original from poisson distribution
#distribution parameter (lambda) is computed from sample
boot_mean = boot_data.mean(axis = 0) #mean of each sample

#plot results
plt.hist(boot_mean, label = 'bootstrap means')
plt.plot([data_mean,data_mean], plt.gca().get_ylim(), color = 'lime', lw = 3, label = 'original data mean')
plt.xlabel('Means of bootstrap samples', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.title('Parametric Bootstrap - Pois(3)', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig("spikes_param_boot.png", bbox_inches='tight')
files.download("spikes_param_boot.png")

In [ ]:
# Clearly incorrect distribution (uniform descrete)

num_boot = 2000
boot_data = np.random.randint(spikes_data.min(), spikes_data.max(), size = (data_len,num_boot) )
boot_mean = boot_data.mean(axis = 0)

plt.hist(boot_mean, label = 'bootstrap means')
plt.plot([data_mean,data_mean], plt.gca().get_ylim(), color = 'lime', lw = 3, label = 'original data mean')
plt.xlabel('Means of bootstrap samples', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.title('parametric bootstrap - Uniform([0 13])', fontsize = 14)
plt.legend(loc = 'upper left')

plt.savefig("spikes_wrong_param_boot.png", bbox_inches='tight')
files.download("spikes_wrong_param_boot.png")


#### Non Parametric Bootstrap

In [ ]:
# non-parametric bootstrap
num_boot = 2000
boot_data = np.random.choice(spikes_data, size = (data_len,num_boot) , replace=True) # resample from data with repetitions
boot_mean = boot_data.mean(axis = 0)

plt.hist(boot_mean, label = 'bootstrap means')
plt.plot([data_mean,data_mean], plt.gca().get_ylim(), color = 'lime', lw = 3, label = 'original data mean')
plt.xlabel('Means of bootstrap samples', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.title('Non-parametric bootstrap')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("spikes_nonparam_boot.png", bbox_inches='tight')
files.download("spikes_nonparam_boot.png")

#### Bootstrap Confidence Intervals

In [ ]:
# parametric bootstrap + CI
num_boot = 2000 #number of bootstrap repititions
boot_data = np.random.poisson(data_mean, size = (data_len, num_boot) ) #select the same sample size as the original from poisson distribution
#distribution parameter (lambda) is computed from sample
boot_mean = boot_data.mean(axis = 0) #mean of each sample

conf = 0.92
ci92 = np.percentile(boot_mean, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[4, 96]

#plot results
ax = plt.subplot(1,1,1)

plt.hist(boot_mean, label = 'Bootstrap Means')
ys = plt.gca().get_ylim()

plt.plot([data_mean,data_mean], ys, color = 'lime', lw = 3, label = 'Original Data Mean')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')

plt.xlabel('Means of Bootstrap Samples', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.title('Parametric Bootstrap - Poisson(3)', fontsize = 14)
handles, labels = ax.get_legend_handles_labels()
handles.pop(2) , labels.pop(2)

plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig("spikes_param_boot_CI.png", bbox_inches='tight')
files.download("spikes_param_boot_CI.png")

In [ ]:
# wrong distribution (uniform descrete) + CI

num_boot = 2000
boot_data = np.random.randint(spikes_data.min(), spikes_data.max(), size = (data_len,num_boot) )
boot_mean = boot_data.mean(axis = 0)

ci92 = np.percentile(boot_mean, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[4, 96]

ax = plt.subplot(1,1,1)

plt.hist(boot_mean, label = 'Bootstrap Means')
ys = plt.gca().get_ylim()

plt.plot([data_mean,data_mean], ys, color = 'lime', lw = 3, label = 'Original Data Mean')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')

plt.xlabel('Means of Bootstrap Samples', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
handles, labels = ax.get_legend_handles_labels()
handles.pop(2) , labels.pop(2)

plt.legend(handles, labels, loc = 'upper left', fontsize = 12)

plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.title('Parametric Bootstrap - Uniform([0 13])', fontsize = 14)

plt.savefig("spikes_wrong_param_boot_CI.png", bbox_inches='tight')
files.download("spikes_wrong_param_boot_CI.png")

### Comprehension Example

In [ ]:
#Creating data and computing effect size
comprehension = [[5,6,8,12,11,9,7,6,8,7,8,12,5,6,7,8,11,7,7,5,3,4,10,6,6,5,7,6,5,6],
                 [6,7,9,13,12,10,11,8,10,9,8,13,6,14,6,11,12,10,9,4,5,12,11,2,7,8,7,6,6,7]]

group1 = np.array(comprehension[0])
group2 = np.array(comprehension[1])

sample1_mean = group1.mean()
sample2_mean = group2.mean()
sample1_var = group1.var(ddof=1) #unbiased estimator
sample2_var = group2.var(ddof=1)

sd_comb = np.sqrt(((group1.shape[0]-1)*sample1_var + (group2.shape[0]-1)*sample2_var)/(group1.shape[0] + group2.shape[0] - 2))
effect_size = (sample2_mean-sample1_mean)/sd_comb
print(effect_size)

#### Unpaired Bootstrap

In [ ]:
# bootstrap unpaired (unmatched pairs)

mean_group1 = group1.mean()
mean_group2 = group2.mean()
mean_diff = mean_group2-mean_group1

plt.scatter(np.ones(len(group1),),group1)
plt.scatter(np.full((len(group2),),2),group2)
plt.xlim((0,3))
plt.ylabel('Score', fontsize = 14)
plt.xticks([1,2], ['Age 3' , 'Age 5'])
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("two_groups_unmatched.png", bbox_inches='tight')
files.download("two_groups_unmatched.png")

num_boot = 2000

boot_group1 = np.random.choice(group1,size = (len(group1),num_boot)) #resample 3 year olds
boot_group2 = np.random.choice(group2,size = (len(group1),num_boot)) #resample 5 year olds

boot_mean1 = boot_group1.mean(axis = 0) #compute means
boot_mean2 = boot_group2.mean(axis = 0)

plt.hist(boot_mean1, alpha = 0.6, label = 'bootsrap samples of 3 year old means')
plt.hist(boot_mean2, alpha = 0.6, label = 'bootsrap samples of 5 year old means')
plt.xlabel('Score', fontsize = 14)
plt.ylabel('Count', fontsize = 14)


plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("hist_boot.png", bbox_inches='tight')
files.download("hist_boot.png")

plt.savefig("bootdists.png", bbox_inches='tight')
files.download("bootdists.png")

plt.figure()
boot_diff_means = boot_mean2-boot_mean1 #compute difference between means

plt.hist(boot_diff_means, label = 'Difference of boot means')
plt.plot([mean_diff, mean_diff], plt.gca().get_ylim(), color = 'lime', lw = 3,
         label = 'Original data difference of means')
plt.xlabel('Differences of Means', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.legend(loc = 'upper left')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("boot_unmatched.png", bbox_inches='tight')
files.download("boot_unmatched.png")

#### Paired Bootstrap

In [ ]:
# bootstrap paired (matched pairs)
fig = plt.figure()
plt.scatter(np.arange(1,len(group1)+1,1),group1, label = 'Age 3')
plt.scatter(np.arange(1,len(group2)+1,1),group2, label = 'Age 5')
plt.xlabel('Child', fontsize = 14)
plt.ylabel('Score', fontsize = 14)
plt.legend(fontsize = 12)
plt.xticks(np.arange(1, len(group1)+1), fontsize = 12)
plt.yticks(fontsize = 12)
fig.set_size_inches(10, 8)

plt.savefig("two_groups_matched.png", bbox_inches='tight')
files.download("two_groups_matched.png")

data_diff = group2-group1 #compute difference between each child at ages 3 and 5

boot_diff = np.random.choice(data_diff, size = (len(group1),num_boot)) #resample the differences

boot_mean_diff = boot_diff.mean(axis = 0) #then compute mean of differences

plt.hist(boot_mean_diff,  label = 'Means of boot differences')
plt.plot([mean_diff, mean_diff], plt.gca().get_ylim(), color = 'lime', lw = 3, label = 'Original data mean of difference')
plt.xlabel('Means of differences', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.ylabel('Count', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("boot_matched.png", bbox_inches='tight')
files.download("boot_matched.png")

#### With Confidence Intervals

In [ ]:
# bootstrap unpaired + CI

num_boot = 2000

boot_group1 = np.random.choice(group1,size = (len(group1),num_boot)) #resample 3 year olds
boot_group2 = np.random.choice(group2,size = (len(group1),num_boot)) #resample 5 year olds

boot_mean1 = boot_group1.mean(axis = 0) #compute means
boot_mean2 = boot_group2.mean(axis = 0)

boot_diff_means = boot_mean2-boot_mean1 #compute difference between means
conf = 0.92

ci92 = np.percentile(boot_diff_means, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[4, 96]

ax = plt.subplot(1,1,1)

plt.hist(boot_diff_means, label = 'Bootstrap Samples')
ys = plt.gca().get_ylim()

plt.plot([mean_diff, mean_diff], ys, color = 'lime', lw = 3,
         label = 'Original data difference of means')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')

plt.xlabel('Bootstrapped Differences of Means', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
handles, labels = ax.get_legend_handles_labels()
handles.pop(2) , labels.pop(2)

plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)


plt.savefig("boot_unmatched_CI.png", bbox_inches='tight')
files.download("boot_unmatched_CI.png")

In [ ]:
# bootstrap paired + CI

data_diff = group2-group1 #compute difference between each child at ages 3 and 5

boot_diff = np.random.choice(data_diff, size = (len(group1),num_boot)) #resample the differences

boot_mean_diff = boot_diff.mean(axis = 0) #then compute mean of differences

ci92 = np.percentile(boot_mean_diff, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[4, 96]


ax = plt.subplot(1,1,1)

plt.hist(boot_mean_diff,  label = 'Bootstrap Samples')
ys = plt.gca().get_ylim()
plt.plot([mean_diff, mean_diff], ys, color = 'lime', lw = 3, label = 'Original data mean of difference')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')

plt.xlabel('Bootstrapped Means of differences', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.ylabel('Count', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim([-1, 4])
handles, labels = ax.get_legend_handles_labels()
handles.pop(2) , labels.pop(2)

plt.legend(handles, labels, loc = 'upper left', fontsize = 12)

plt.savefig("boot_matched_CI.png", bbox_inches='tight')
files.download("boot_matched_CI.png")

### Bootstrap Group Comparison

Upload data (file in tutorial folder)

In [ ]:
# Upload file for comaprision between two groups
uploaded = files.upload()

data = pd.read_csv(io.BytesIO(uploaded['kid_calories.csv']))

for col in data.columns: # see names of the columns in a loop
    print(col)

print(data)

Look at data

In [ ]:
# Examine data
#histogram

group_names = np.unique(data.group)
print(group_names)
group1 = data.calories[data.group == group_names[0]]
group2 = data.calories[data.group == group_names[1]]

group1_mean = group1.mean()
group2_mean = group2.mean()

bin_edges = np.arange(100,700,50)

plt.hist(group1,bins = bin_edges, alpha = 0.6, label = 'Kid cooks')
plt.hist(group2, bins = bin_edges, alpha = 0.6, label = 'Parent cooks')
ys = plt.gca().get_ylim()

plt.plot([group1_mean,group1_mean], ys, color = 'dodgerblue',
         lw = 3, label = 'Kid cooks mean')
plt.plot([group2_mean,group2_mean],ys,color = 'darkorange',
         lw=3,  label = 'Parent cooks mean')
plt.xlabel('Calories [kcal]', fontsize = 14)
plt.ylabel('Number of Kids', fontsize = 14)
plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("hist_cook.png", bbox_inches='tight')
files.download("hist_cook.png")

In [ ]:
# boxplot

plt.boxplot([group1, group2] , positions = np.array((1,2)) )

plt.xticks((1,2) , ('kid Cooks' , 'Parent Cooks'), fontsize = 12)
plt.yticks(fontsize = 12)
plt.ylabel('Calories [kcal]', fontsize = 14)
plt.xlabel('Group', fontsize = 14)

plt.savefig("box_cook.png", bbox_inches='tight')
files.download("box_cook.png")

#### Bootstrap

In [ ]:
#non-parametric non-paired bootstrap

num_boots = 2000
boot_samples1 = np.random.choice(group1, size = (len(group1),num_boots))
boot_samples2 = np.random.choice(group2, size = (len(group2),num_boots))

boot_samples1_means = boot_samples1.mean(axis = 0)
boot_samples2_means = boot_samples2.mean(axis = 0)

plt.hist(boot_samples1_means, alpha = 0.6, label = 'boot samples means of kids cook')
plt.hist(boot_samples2_means, alpha = 0.6, label = 'boot samples means of parents cook')
plt.xlabel('Mean of Calories [kcal]', fontsize = 14)
plt.ylabel('Count', fontsize = 14)


plt.legend(loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig("hist_boot.png", bbox_inches='tight')
files.download("hist_boot.png")

In [ ]:
# boostrap difference of means

boot_samples_diff_means = boot_samples1_means-boot_samples2_means

orig_diff_mean = group1_mean-group2_mean

# confidence interval (CI) 92%
conf = 0.92
ci92 = np.percentile(boot_samples_diff_means, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[2.5, 97.5]

print('92% Confidence Interval = (' + str(ci92[0]) + ' , ' + str(ci92[1]) + ')')

ax = plt.subplot(1,1,1)
plt.hist(boot_samples_diff_means, label = 'Bootstrap difference of means')
ys = plt.gca().get_ylim()
plt.plot([orig_diff_mean,orig_diff_mean],ys,
         color = 'lime', lw = 3, label = 'Original samples difference of means')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')
#plt.plot([0,0],ys,color = 'black', lw = 2, linestyle = '--', label = 'zero')
handles, labels = ax.get_legend_handles_labels()
handles.pop(3) , labels.pop(3)
plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('Mean of Calories [kcal]', fontsize = 14)
plt.ylabel('Count', fontsize = 14)

plt.savefig("ci.png", bbox_inches='tight')
files.download("ci.png")


In [ ]:
#compare paremetric and bootstrapped distributions
plt.figure()
plt.hist(boot_samples1_means, label = 'Bootstrap Means of Kid Group')
plt.plot([group1_mean,group1_mean], plt.gca().get_ylim(),
         color = 'lime', lw = 3, label = 'Original Sample Mean')
plt.title('Bootstrap Distribution', fontsize = 14)
plt.xlabel('Mean of Calories [kcal]', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.xlim([350, 500])
plt.savefig("bootstrap.png", bbox_inches='tight')
files.download("bootstrap.png")

ParametricSample = np.random.normal(group1_mean, group1.std(ddof = 1)/(group1.shape[0]**0.5), (boot_samples1_means.shape[0], 1))
plt.figure()
plt.hist(ParametricSample, label = 'Sampled Means from Normal Distribution')
plt.plot([group1_mean,group1_mean],plt.gca().get_ylim(),
         color = 'lime', lw = 3, label = 'Original Sample Mean')
plt.title('Sample from Normal Distribution N(mean(sample), std(sample)/sqrt(N))', fontsize = 14)
plt.xlabel('Mean of Calories [kcal]', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim([350, 500])
plt.savefig("distribution.png", bbox_inches='tight')
files.download("distribution.png")

In [ ]:
# add non parametric and parametric confidence intervals
conf = 0.92
ci92 = np.percentile(boot_samples1_means, [100*(1-conf)/2,100*(1-(1-conf)/2)]) #[4, 96]
ax = plt.subplot(1,1,1)
plt.hist(boot_samples1_means, label = 'Bootstrap Means')
ys = plt.gca().get_ylim()
plt.plot([group1_mean,group1_mean],ys,
         color = 'lime', lw = 3, label = 'Original Samples Mean')
plt.plot([ci92,ci92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(ci92[0], 2)}, {round(ci92[1], 2)})')
handles, labels = ax.get_legend_handles_labels()
handles.pop(3) , labels.pop(3)
plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('Mean of Calories [kcal]', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xlim([350, 500])
plt.savefig("cin.png", bbox_inches='tight')
files.download("cin.png")

N = group1.shape[0]
group1_se = group1.std(ddof = 1)/(N**0.5)
t_ci92 = stats.t.ppf([0.04,0.96], N - 1)
CI92 = group1_mean + group1_se*t_ci92

plt.figure()
ax = plt.subplot(1,1,1)
plt.hist(ParametricSample, label = 'Sampled Means from Normal Distribution')
plt.plot([group1_mean,group1_mean],plt.gca().get_ylim(),
         color = 'lime', lw = 3, label = 'Original Sample Mean')
plt.plot([CI92,CI92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(CI92[0], 2)}, {round(CI92[1], 2)})')
handles, labels = ax.get_legend_handles_labels()
handles.pop(3) , labels.pop(3)
plt.legend(handles, labels, loc = 'upper left', fontsize = 12)

plt.title('Sample from Normal Distribution N(mean(sample), std(sample)/sqrt(N))', fontsize = 14)
plt.xlabel('Mean of Calories [kcal]', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlim([350, 500])

plt.savefig("cit.png", bbox_inches='tight')
files.download("cit.png")

### Auditory - Visual Example

Load data (file in the tutorial folder)

In [ ]:
uploaded = files.upload()

data = pd.read_csv(io.BytesIO(uploaded['AudioVisual.csv']))

for col in data.columns: # see names of the columns in a loop
    print(col)

print(data)

Visualizations

In [ ]:
# histogram by stimulus or group
fig = plt.figure()

plt.subplot(1, 2, 1)
sns.histplot(data=data, x = 'ResponseTime', hue = 'Stimulus')
plt.xlabel('Response Time [msec]', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.subplot(1, 2, 2)
sns.histplot(data=data, x = 'ResponseTime', hue = 'Group', palette = ['C2', 'C3'])
fig.subplots_adjust(bottom=0.1, right=1.5, top=0.9)
plt.xlabel('Response Time [msec]', fontsize = 14)
plt.ylabel('Count', fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

fig.set_size_inches((7, 5))

plt.savefig("sephist.png", bbox_inches='tight')
files.download("sephist.png")

In [ ]:
# boxplot

fig, ax = plt.subplots(1,3)
sns.boxplot(ax=ax[0], data=data, x = 'Stimulus', y = 'ResponseTime', hue = 'Stimulus', palette = {'auditory':'blue', 'visual':'orange'})

sns.boxplot(ax=ax[1], data=data, x = 'Group', y = 'ResponseTime', hue = 'Group', palette = {'NonMusician':'g', 'Musician':'r'})

stim_group = ['auditory musician']*len(data.ResponseTime)
data['stim_group'] = stim_group
data.loc[(data.Stimulus==np.unique(data.Stimulus)[0]) & (data.Group==np.unique(data.Group)[1]), 'stim_group'] = 'auditory nonmusician'
data.loc[(data.Stimulus==np.unique(data.Stimulus)[1]) & (data.Group==np.unique(data.Group)[0]), 'stim_group'] = 'visual musician'
data.loc[(data.Stimulus==np.unique(data.Stimulus)[1]) & (data.Group==np.unique(data.Group)[1]), 'stim_group'] = 'visual nonmusician'


sns.boxplot(ax=ax[2], data=data, x = 'stim_group', y = 'ResponseTime', hue = 'stim_group',
            palette = {'auditory musician':'cyan', 'auditory nonmusician':'brown', 'visual musician': 'orchid', 'visual nonmusician': 'teal'})
ax[2].set_xticklabels(['Auditory \n Not Musician' , 'Auditory \n Musician' , 'Visual \n Not Musician' , 'Visual \n Musician'])
ax[2].set_xlabel('Stimulus&Group')

ax[0].set_ylabel('Response Time [msec]')
ax[1].set_ylabel('Response Time [msec]')
ax[2].set_ylabel('Response Time [msec]')

fig.subplots_adjust(bottom=0.1, right=2.2, top=0.9)


plt.savefig("box.png", bbox_inches='tight')
files.download("box.png")

In [ ]:
# violinplot

fig, ax = plt.subplots(1,3)
sns.violinplot(ax=ax[0], data=data, x = 'Stimulus', y = 'ResponseTime', hue = 'Stimulus', palette = {'auditory':'blue', 'visual':'orange'})

sns.violinplot(ax=ax[1], data=data, x = 'Group', y = 'ResponseTime', inner = 'stick', hue ='Group',
               palette = {'NonMusician':'g', 'Musician':'r'})

sns.violinplot(ax=ax[2], data=data, x = 'stim_group', y = 'ResponseTime', inner = 'stick', hue = 'stim_group',
            palette = {'auditory musician':'cyan', 'auditory nonmusician':'brown', 'visual musician': 'orchid', 'visual nonmusician': 'teal'})

ax[2].set_xticklabels(['Auditory \n Not Musician' , 'Auditory \n Musician' , 'Visual \n Not Musician' , 'Visual \n Musician'])
ax[2].set_xlabel('Stimulus & Group')


ax[0].set_ylabel('Response Time [msec]')
ax[1].set_ylabel('Response Time [msec]')
ax[2].set_ylabel('Response Time [msec]')

fig.subplots_adjust(bottom=0.1, right=1.8, top=0.9)

plt.savefig("violin.png", bbox_inches='tight')
files.download("violin.png")

#### Confidence Intervals

In [ ]:
aud = np.array(data.ResponseTime[data.Stimulus=='auditory'])
vis = np.array(data.ResponseTime[data.Stimulus=='visual'])

m1 = np.mean(aud)
m2 = np.mean(vis)

s1 = np.std(aud, ddof = 1)
s2 = np.std(vis, ddof = 1)

N1 = len(aud)
N2 = len(vis)

#auditory
t_ci92 = stats.t.ppf([0.04,0.96], N1 - 1)
CI92 = m1 + (s1/N1**0.5)*t_ci92
ParametricSample = np.random.normal(m1, s1/(N1**0.5), (100000, 1))
plt.figure()
ax = plt.subplot(1,1,1)
plt.hist(ParametricSample, bins = 1000, label = 'Sampled Means from Normal Distribution')
ys = plt.gca().get_ylim()
plt.plot([m1, m1], ys, color = 'lime', lw = 3, label = 'Original Sample Mean')
plt.plot([CI92,CI92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(CI92[0], 2)}, {round(CI92[1], 2)})')
handles, labels = ax.get_legend_handles_labels()
handles.pop(3) , labels.pop(3)
plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.title('Sample from Normal Distribution for Auditory', fontsize = 14)
plt.xlabel('Auditory Mean Response Time [msec]', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig("aud.png", bbox_inches='tight')
files.download("aud.png")

#auditory
CI92 = m2 + (s2/N2**0.5)*t_ci92
ParametricSample = np.random.normal(m2, s2/(N2**0.5), (100000, 1))
plt.figure()
ax = plt.subplot(1,1,1)
plt.hist(ParametricSample, bins = 1000, label = 'Sampled Means from Normal Distribution')
ys = plt.gca().get_ylim()
plt.plot([m2, m2], ys, color = 'lime', lw = 3, label = 'Original Sample Mean')
plt.plot([CI92,CI92], ys ,color = 'red', lw = 3, label = f'92% CI = ({round(CI92[0], 2)}, {round(CI92[1], 2)})')
handles, labels = ax.get_legend_handles_labels()
handles.pop(3) , labels.pop(3)
plt.legend(handles, labels, loc = 'upper left', fontsize = 12)
plt.title('Sample from Normal Distribution for Visual', fontsize = 14)
plt.xlabel('Visual Mean Response Time [msec]', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig("vis.png", bbox_inches='tight')
files.download("vis.png")

### Confidence Intervals of Regression Coefficients

Loading data for this example (in folder)

In [ ]:
#blood pressure
#upload and read from PC/Drive
# after running press 'Choose File'

uploaded = files.upload()

data = pd.read_csv(io.BytesIO(uploaded['bp.csv']))

for col in data.columns: # see names of the columns in a loop
    print(col)

print(data)

In [ ]:
# presentation

age = data.age
bp = data.bp

plt.scatter(age,bp)
plt.xlabel('Age', fontsize = 14)
plt.ylabel('Blood Pressure', fontsize = 14)
plt.title('Age Vs. Blood Pressure', fontsize = 16)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)


plt.savefig("scatter_bp.png", bbox_inches='tight')
files.download("scatter_bp.png")

Regression + presentation of regression

In [ ]:
#regression using sklearn
x = np.reshape(age, (len(age), 1))
y = np.reshape(bp, (len(bp), 1))
regr_model = LinearRegression().fit(x, y)
b0 = regr_model.intercept_
b1 = regr_model.coef_
print(f'b0 = {np.round(b0, 4)}, b1 = {np.round(b1, 4)}')


yhat = b0+b1*x

plt.scatter(x,y)
plt.plot(x,yhat, color = 'lime')
plt.xlabel('Age', fontsize = 14)
plt.ylabel('Blood Pressure', fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.title(f'blood pressure = {np.round(b0[0],2)}+{np.round(b1[0][0],2)}$\cdot$age')
plt.savefig("bp_reg.png", bbox_inches='tight')
files.download("bp_reg.png")

Caclulations for the confidence intervals

In [ ]:
#calculate sum of squares
N = len(x)
SSres = np.dot(np.squeeze(y-yhat),np.squeeze(y-yhat))

print(f'SSres = {SSres}')

SSreg = np.dot(np.squeeze(yhat-y.mean()),np.squeeze(yhat-y.mean()))
print(f'SSreg = {SSreg}')

#calculate Syx
Syx2 = SSres/(N-2)
print(f'Syx2 = {Syx2}')

#calculate variance of coefficients
X = np.squeeze(np.array([np.ones_like(x), x])).T
Sb2 = Syx2*np.linalg.inv(X.T@X)*np.eye(2)

print(Sb2)

Sb = Sb2**0.5
print(Sb)

And the confidence intervals themselves

In [ ]:
#confidence intervals for coefficients
CI_b0 = b0+Sb[0,0]*stats.t.ppf([0.04,0.96],N-2)
CI_b1 = b1+Sb[1,1]*stats.t.ppf([0.04,0.96],N-2)

print(f'CI for b0 = {CI_b0}')
print(f'CI for b1 = {CI_b1}')